In [ ]:
from dataclasses import dataclass
from __future__ import annotations

@dataclass
class Dir:
    path: str
    children: list[AoCFile|Dir]
    parent: Dir

@dataclass
class AoCFile:
    name: str
    size: int

In [ ]:
import re

cd_pattern = "\$ cd (?P<directory>.*)"
ls_pattern = "\$ ls"
dir_pattern = "dir (?P<dir>.*)"
size_pattern = "(?P<size>\d+) (?P<filename>.*)"


def shell():
    curr_dir = None
    for line in open("input.txt"):
        if (dir := re.match(cd_pattern, line)):
            directory = dir.group("directory")
            if directory == "..":
                curr_dir = curr_dir.parent
            else:
                tmp = Dir(dir.group("directory"), [], curr_dir)
                if curr_dir:
                    curr_dir.children.append(tmp)
                curr_dir = tmp
        elif re.match(dir_pattern, line):
            pass 
        elif (file := re.match(size_pattern, line)):
            val = int(file.group("size"))
            curr_dir.children.append(AoCFile(file.group("filename"), val))
    
    while curr_dir and curr_dir.parent:
        curr_dir = curr_dir.parent
    return curr_dir


In [ ]:

def iter_fs(node):
    if isinstance(node, AoCFile):
        yield node
    elif isinstance(node, Dir):
        yield node
        for child in node.children:
            yield from iter_fs(child)


def iter_directories(node):
    return [x for x in iter_fs(node) if isinstance(x, Dir)]

def sum_directory(node):
    return sum([child.size for child in node.children if isinstance(child, AoCFile)]) \
        + sum([sum_directory(child) for child in node.children if isinstance(child, Dir)])

root = shell()
sum([val for d in iter_directories(root) if (val := sum_directory(d)) < 100000])


In [ ]:
total_size = 70000000 
required_size = 30000000
used_size = sum_directory(shell())
space_left = total_size - used_size
extra_space_required = required_size - space_left
extra_space_required

sorted([(dir_size, d.path) for d in iter_directories(root) if (dir_size := sum_directory(d)) > extra_space_required])[0]